In [180]:
import urllib
import json
import unicodedata
import requests
import bs4
import os
import io
import time
from IPython.display import clear_output

In [188]:
def get_all_pokemon_base_entries():
    base_page_url = "https://bulbapedia.bulbagarden.net/wiki/List_of_Pokémon_by_National_Pokédex_number"
    base_page_content = requests.get(base_page_url).content
    root = bs4.BeautifulSoup(base_page_content, "html5lib").find(id="mw-content-text")

    pokemon_base_entries = {}
    generation_table_tags = root.find_all("table")[1:8]

    for gen_table_tag in generation_table_tags:
        pokemon_tr_tags = gen_table_tag.find("tbody").find_all("tr")[1:]

        for tr_tag in pokemon_tr_tags:
            pokemon_name, number, wikilink = get_pokemon_name_and_wikilink(tr_tag)
            pokemon_base_entries[pokemon_name] = {"wikilink": wikilink, "pokedex_number": number}

    return pokemon_base_entries


def get_pokemon_name_and_wikilink(tr_tag):
    cols = tr_tag.find_all("td", recursive=False)

    number = int(cols[1].string.replace(" ", "")[1:-1])

    details_link_tag = cols[3].find_all("a", recursive=False)[0]
    details_url = "https://bulbapedia.bulbagarden.net" + details_link_tag["href"]

    base_name = details_link_tag.string
    
    return base_name, number, details_url

In [189]:
pokemon_list = get_all_pokemon_base_entries()

In [215]:
baseurl = "http://bulbapedia.bulbagarden.net/w/api.php?"
action = "action=query"
content = "prop=revisions"
rvprop ="rvprop=content"
dataformat = "format=json"

dir_path = "./pokemon_files_raw"
def create_pokemon_file():
    counter = 0
    for name, info in pokemon_list.items():

        title = "titles=" + name + "_(Pok%C3%A9mon)"
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
            print('created', dir_path)

        pokedex_number = info["pokedex_number"]
        file_path = f"{dir_path}/[{pokedex_number}]{name}"
        if os.path.isfile(file_path):
            continue
        query = "{}{}&{}&{}&{}&{}".format(baseurl, action, title, content, rvprop, dataformat)
        print(query)
        pokemon_text = requests.get(query).content.decode("utf-8")
        pokemon_text = json.loads(pokemon_text)
        pokemon_text = pokemon_text["query"]["pages"]
        
        pokemon_page_number = next(iter(pokemon_text))
        pokemon_text = pokemon_text[pokemon_page_number]["revisions"][0]

        with io.open(file_path, 'w', encoding='utf-8') as f:
            f.write(u''+ str(pokemon_text))
            f.close()
        counter +=1
        clear_output(wait=True)
        print(f"{int(pokedex_number)} pokemon files created. done. {len(pokemon_list.keys()) - int(pokedex_number)} left")
        


In [216]:
create_pokemon_file()

809 pokemon files created. done. 0 left
